In [1]:
import pycuda.driver as cuda
from facenet_config import InceptionResnetV1, fixed_image_standardization
from facenet_capstone.utils import training
# from utils.logits import CosFace
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.optim import lr_scheduler
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import torch.optim as optim
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Check current directory
cwd = os.getcwd()
print('\n***** *****\n')
print(cwd)

# Determine if an nvidia GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# For reproducibility, Seed the RNG for all devices (both CPU and CUDA):
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
# np.random.seed(0)
# random.seed(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print('\n***** *****\n')
print('Running on device: {}'.format(device)) 

# ***** *****

# Set training parameters
batch_size = 128
epochs = 40
workers = 0 if os.name == 'nt' else 8 #nt for Windows
print(workers)


***** *****

C:\Users\DS\Yonsei_Capstone

***** *****

Running on device: cuda:0
0


In [2]:
# Locate training and testing folders
train_dir = 'D:/TinyFace_Closed/train'
test_dir = 'D:/TinyFace_Closed/test'

train_trans = transforms.Compose( [ transforms.Resize((160,160)),
                                    transforms.RandomHorizontalFlip(p=0.6), 
                                    np.float32, transforms.ToTensor(), fixed_image_standardization ] )

test_trans = transforms.Compose( [ transforms.Resize((160,160)),
                                    # transforms.RandomHorizontalFlip(p=0.6), 
                                    np.float32, transforms.ToTensor(), fixed_image_standardization ] )

train_set = datasets.ImageFolder(train_dir, transform = train_trans)
test_set = datasets.ImageFolder(test_dir, transform = test_trans)

train_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size, shuffle = True, num_workers = workers)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size, shuffle = False, num_workers = workers)

assert(train_set.class_to_idx.items() == test_set.class_to_idx.items()) #assertionError if !=

class_names = train_set.classes

# print(train_set.class_to_idx)
# print(train_set.class_to_idx.keys())
# print(train_set.class_to_idx.values())
# print(train_set.class_to_idx.items())
# print(test_set.class_to_idx.items())


In [18]:
# Load pre-learned InceptionResnetV1 
print('\n***** *****\n')
print('Loading Pre-Learned InceptionResnetV1 ( Pre-Trained w.r.t. VGGFace2 ) ... ' )  
print()

resnet = InceptionResnetV1(pretrained = 'vggface2', classify=True, num_classes=len(class_names))

# print state_dict of the model 
'''
print("resnet's state_dict:")
for param_tensor in resnet.state_dict():
    print(param_tensor, "\t", resnet.state_dict()[param_tensor].size())
'''
# resnet.logits = CosFace(in_features = resnet.last_linear.out_features, out_features = len(class_names))
resnet.to(device)

# print(resnet)
# print(len(class_names))

# ***** *****

# Resnet : Determine parameters to be freezed, or unfreezed
# Retrain only logit >>> softmax classifier
for name, param in resnet.named_parameters():
    # if name in ['last_linear.weight', 'last_linear.bias', 'last_bn.weight', 'last_bn.bias']:
    if name in ['logits.weight', 'logits.bias']:
        param.requires_grad = True
    else:
        param.requires_grad = False #False if only the last fc layer
        
# Resnet : Display all learnable parameters
print()
print('Learnable Paramater List')
print('-' *24)
print()

for name, param in resnet.named_parameters():
    if param.requires_grad:
        print('resnet\t:', name)

# ***** *****

# Freeze all batch normalization (BN) parameters
for name, layer in resnet.named_modules():
    if isinstance(layer,torch.nn.BatchNorm2d):  
        layer.weight.requires_grad = False
        layer.bias.requires_grad = False
 


***** *****

Loading Pre-Learned InceptionResnetV1 ( Pre-Trained w.r.t. VGGFace2 ) ... 


Learnable Paramater List
------------------------

resnet	: logits.weight
resnet	: logits.bias


In [19]:
# Set lr, lr_scheduler, weight_decay
lr = 0.001
milestones = [12, 24, 36, 48]
weight_decay = 0.0000

# ***** *****
# Set an optimizer, scheduler, etc.
net_params = resnet.parameters()
optimizer = optim.Adam(net_params, lr = lr, weight_decay = weight_decay)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones = milestones, gamma = 0.1)

# ***** *****

# Define loss and evaluation functions
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [20]:
# Start fine-tuning resnset
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

# ***** ***** print initial evaluation output

print('\n\nInitial')
print('-' * 8)

resnet.eval()

training.run_training(resnet, loss_fn, test_loader,
                     batch_metrics=metrics, show_running=True, device=device,
                     writer=writer)

# ***** ***** training mode

for epoch in range(epochs):
    
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    
    for layer in resnet.modules():
        if isinstance(layer,torch.nn.modules.batchnorm.BatchNorm2d):
            #print(layer)
            layer.eval()
    
    training.run_training(resnet, loss_fn, train_loader, optimizer, scheduler, #training
                          batch_metrics=metrics, show_running=True, device=device,
                          writer=writer)
    
    # ***** evaluation mode

    resnet.eval()

    training.run_training(resnet, loss_fn, test_loader,
                         batch_metrics=metrics, show_running=True, device=device,
                         writer=writer)

# ***** *****

writer.close()



Initial
--------
Test |    30/30   | loss:    8.7018 | fps:  559.3226 | acc:    0.0013   

Epoch 1/40
----------
Train |    96/96   | loss:    7.7878 | fps:  576.1533 | acc:    0.0587   
Test |    30/30   | loss:    6.8420 | fps:  710.9582 | acc:    0.0807   

Epoch 2/40
----------
Train |    96/96   | loss:    5.1272 | fps:  699.7684 | acc:    0.2488   
Test |    30/30   | loss:    5.7310 | fps:  706.7406 | acc:    0.1802   

Epoch 3/40
----------
Train |    96/96   | loss:    3.4506 | fps:  697.0641 | acc:    0.4933   
Test |    30/30   | loss:    5.0510 | fps:  709.7490 | acc:    0.2628   

Epoch 4/40
----------
Train |    96/96   | loss:    2.3915 | fps:  692.6949 | acc:    0.6776   
Test |    30/30   | loss:    4.6289 | fps:  715.3400 | acc:    0.3044   

Epoch 5/40
----------
Train |    96/96   | loss:    1.7347 | fps:  694.5745 | acc:    0.7774   
Test |    30/30   | loss:    4.4151 | fps:  723.3185 | acc:    0.3240   

Epoch 6/40
----------
Train |    96/96   | loss:    1.367

In [25]:
path_weight = 'C:/Users/DS/Yonsei_Capstone/save_model/resnet_1.pt'
path_all = 'C:/Users/DS/Yonsei_Capstone/save_model/resnet.tar'
torch.save(resnet.state_dict(),path_weight)
torch.save(resnet,path_all)

In [ ]:
print("-----")